In [7]:
import gradio as gr
 
def compare_images(uploaded_image):
   
    accuracy = 0.85
    return f"Similarity Accuracy: {accuracy * 100}%"
 
# Gradio interface
with gr.Blocks() as demo:
    # with gr.Row():
        # with gr.Column():
            # image_input = gr.Image(type="pil", label="Upload your image")
            # submit_button = gr.Button("Submit")
        # with gr.Column():
            # result_output = gr.Textbox(label="Result")

    with gr.Tab("Image"):
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload your image")
            submit_button = gr.Button("Submit")
    with gr.Tab("Text"):
        with gr.Column():
            result_output = gr.Textbox(label="Result")
   
    submit_button.click(fn=compare_images, inputs=image_input, outputs=result_output)
 
demo.launch(share=True)

Running on local URL:  http://127.0.0.1:7866

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


In [26]:
from azure.cognitiveservices.vision.customvision.prediction import CustomVisionPredictionClient
from msrest.authentication import ApiKeyCredentials
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import os
import io
import gradio as gr

# Authenticate client

prediction_endpoint = "https://cvgradio-prediction.cognitiveservices.azure.com"
prediction_key = "7290a12026054f7fa6b49ea9d033222c"
project_id = "6a07261c-a976-4f9c-b3f9-1c0dffdc6fa8"
model_name = "Iteration1"

credentials = ApiKeyCredentials(in_headers={"Prediction-key": prediction_key})
predictor = CustomVisionPredictionClient(endpoint=prediction_endpoint, credentials=credentials)

def predict_image(input_img): 
    h, w, ch = np.array(input_img).shape 
    img_byte_arr = io.BytesIO() 
    input_img.save(img_byte_arr, format='png') 
    img_bytes = img_byte_arr.getvalue() 
    results = predictor.detect_image(project_id, model_name, img_bytes) 

    fig, ax = plt.subplots(figsize=(8, 8)) 
    ax.axis('off') 

    draw = ImageDraw.Draw(input_img) 
    lineWidth = int(w / 100) 
    color = 'magenta' 

    for prediction in results.predictions: 
        if (prediction.probability * 100) > 50: 
            left = prediction.bounding_box.left * w 
            top = prediction.bounding_box.top * h 
            width = prediction.bounding_box.width * w 
            height = prediction.bounding_box.height * h 

            points = ((left, top), (left + width, top), (left + width, top + height), (left, top + height), (left, top)) 
            draw.line(points, fill=color, width=lineWidth) 
            ax.annotate(prediction.tag_name + ' {0:.2f}%'.format(prediction.probability * 100), (left, top), color=color)

    ax.imshow(input_img)

    # 예측 결과 텍스트 생성
    predictions_text = "\n".join(
        f"{prediction.tag_name}: {prediction.probability * 100:.2f}%" 
        for prediction in results.predictions 
        if (prediction.probability * 100) > 50
    )

    greeting_text = "안녕하세요. 사용자님."

    return fig, predictions_text, greeting_text

result_output = gr.Textbox(label="Result")
text_output = gr.Textbox(label="Greeting")

demo = gr.Interface(
    fn=predict_image, 
    inputs=gr.Image(type='pil'), 
    outputs=[gr.Plot(), result_output, text_output]
) 
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7888

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
